In [1]:
import pandas as pd
import numpy as np
from StringIO import StringIO
from zipfile import ZipFile
from urllib import urlopen
from sklearn.linear_model import LogisticRegression
import statsmodels.formula.api as smf

import pandas as pd
import numpy as np
from StringIO import StringIO
from zipfile import ZipFile
from urllib import urlopen
import requests
import csv
import matplotlib.pylab as plt
import seaborn as sns
import pylab as pylab
from collections import OrderedDict
from sklearn.datasets import make_classification
from sklearn.datasets import make_blobs
from matplotlib.pylab import plt
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline

In [2]:
def unzip_read_file(year):
    url = urlopen("http://www.nyc.gov/html/nypd/downloads/zip/analysis_and_planning/{}_sqf_csv.zip".format(year))
    zipfile = ZipFile(StringIO(url.read()))
    try:
        with zipfile.open("{}.csv".format(year)) as file:
            df = pd.read_csv(file)
            
    except: # 2015 csv name is different format than all the rest...go figure.
        with zipfile.open("{}_sqf_csv.csv".format(year)) as file:
            df = pd.read_csv(file)
    return df

In [3]:
df = pd.DataFrame()

In [4]:
# This will capture all data. Loading may take a while and will consume a fair amount of memory. 
# Recommended 16GB of memory if you want to run this cell...

# the rename is there to normalize the data from year to year. without,
# data of different years will not overlap properly

cols = []
#
for year in range(2003, 2016): #change this back to 2016 for all data
    year_df = unzip_read_file(year)
    year_df.rename(columns={'adrnum': 'addrnum', 'adrpct': 'addrpct', 'detailcm': 'detailCM', 
                            'dettyp_c': 'dettypCM', 'dettypcm': 'dettypCM', 'linecm': 'lineCM', 
                            'premtyp': 'premtype', 'prenam': 'prename', 'rescod': 'rescode',
                            'stinter': 'strintr', 'stname': 'strname'}, inplace=True)
    cols.append(list(year_df.columns))
    if len(list(year_df.columns)) != len(set(list(year_df.columns))):
        print sorted(list(year_df.columns))
    print year
    df = df.append(year_df)

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (81,82,83,84,85) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (4,102,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


2003
2004
2005

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (73,74,81,82,83,84,85,107,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (1,2,10,18,19,73,74,105,107) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):



2006
2007

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (4,73,74,81,82,83,84,85,102,109,110) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (4,73,74) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


...


2010
2011

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (73,103) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (10,73,103,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):



2012
2013

/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (10,73,83) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/opt/rh/anaconda/root/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (10,83) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):



2014
2015


In [5]:
# early years missing detailCM, so excluded at this point...

# 'year', 'pct', 'datestop', 'timestop', 'age' , 'perobs'


sqf_primary_select = df.loc[:,['inout', 'explnstp', 'arstmade', 
      'sumissue', 'frisked', 'searched', 'contrabn',  
      'rf_vcrim', 'rf_othsw', 'rf_attir', 'cs_objcs', 'cs_descr', 
      'cs_casng', 'cs_lkout', 'rf_vcact', 'cs_cloth', 'cs_drgtr', 
      'cs_furtv', 'rf_rfcmp', 'rf_verbl', 'cs_vcrim', 'cs_bulge', 
      'cs_other',  'rf_knowl',  'sb_hdobj', 'sb_outln', 'sb_admis', 
      'sb_other', 'rf_furt', 'rf_bulg',  'sex', 'race']] # 'detailCM'

In [6]:
sqf_primary_select2 = df.loc[:,['year', 'inout', 'explnstp', 'arstmade', 
      'sumissue', 'frisked', 'searched', 'contrabn',  
      'rf_vcrim', 'rf_othsw', 'rf_attir', 'cs_objcs', 'cs_descr', 
      'cs_casng', 'cs_lkout', 'rf_vcact', 'cs_cloth', 'cs_drgtr', 
      'cs_furtv', 'rf_rfcmp', 'rf_verbl', 'cs_vcrim', 'cs_bulge', 
      'cs_other',  'rf_knowl',  'sb_hdobj', 'sb_outln', 'sb_admis', 
      'sb_other', 'rf_furt', 'rf_bulg',  'sex', 'race']]

In [7]:

print sqf_primary_select.columns

Index([u'inout', u'explnstp', u'arstmade', u'sumissue', u'frisked',
       u'searched', u'contrabn', u'rf_vcrim', u'rf_othsw', u'rf_attir',
       u'cs_objcs', u'cs_descr', u'cs_casng', u'cs_lkout', u'rf_vcact',
       u'cs_cloth', u'cs_drgtr', u'cs_furtv', u'rf_rfcmp', u'rf_verbl',
       u'cs_vcrim', u'cs_bulge', u'cs_other', u'rf_knowl', u'sb_hdobj',
       u'sb_outln', u'sb_admis', u'sb_other', u'rf_furt', u'rf_bulg', u'sex',
       u'race'],
      dtype='object')


In [8]:
def fix_nans(column):
    if column in [' ', '  ', '']:
        column = np.nan
    
    return column

In [9]:
for i in sqf_primary_select.columns:
    sqf_primary_select[i] = sqf_primary_select[i].apply(fix_nans)
    sqf_primary_select2[i] = sqf_primary_select2[i].apply(fix_nans)

In [10]:
# original data size
len(sqf_primary_select)

5052743

In [11]:
# Dataframe after nan reduction
sqf_primary_select.dropna(inplace=True)
sqf_primary_select2.dropna(inplace=True)

len(sqf_primary_select)

5008717

In [12]:

# Data cleaning to generate numeric values


d = {'Y': 1, 'N': 0}
sqf_primary_select = sqf_primary_select.replace(d)

In [13]:
#sqf_primary_select.head(5)

In [14]:
sqf_primary_select = pd.get_dummies(sqf_primary_select)
#sqf_primary_select.columns

In [15]:
#Linear Regression:
#note that linear regression is not the best form of regression for this data
#we have a dscrete dependent variable (arrests made is a yes or no variable)
#logistic regression (in the second part of this notebook) is better suited to discrete variables
lm = smf.ols(formula = 'arstmade ~ explnstp + ' + ' + '.join(sqf_primary_select.columns[2:]), \
             data = sqf_primary_select).fit()


In [16]:
temp = pd.DataFrame(lm.params)
temp2 = pd.DataFrame(lm.pvalues)
printout = pd.concat([temp, temp2], axis = 1)
printout.columns = ['linear params', 'linearpvalue']
printout['var'] = printout.index
print printout

           linear params   linearpvalue        var
Intercept  -1.183776e+08   6.270085e-01  Intercept
explnstp   -2.176676e-03   1.230802e-01   explnstp
sumissue   -4.943126e-02   0.000000e+00   sumissue
frisked     3.343392e-02   0.000000e+00    frisked
searched    3.892120e-01   0.000000e+00   searched
contrabn    4.123669e-01   0.000000e+00   contrabn
rf_vcrim   -9.187001e-03  4.499973e-222   rf_vcrim
rf_othsw    1.063812e-02  4.530451e-214   rf_othsw
rf_attir   -1.170147e-02  2.002078e-270   rf_attir
cs_objcs    3.688222e-02   0.000000e+00   cs_objcs
cs_descr    1.928218e-02   0.000000e+00   cs_descr
cs_casng   -8.719355e-03   0.000000e+00   cs_casng
cs_lkout   -6.493163e-03  8.681749e-184   cs_lkout
rf_vcact   -9.140523e-04   5.593913e-03   rf_vcact
cs_cloth   -7.292067e-03   6.151215e-72   cs_cloth
cs_drgtr    1.118093e-02   0.000000e+00   cs_drgtr
cs_furtv   -1.034250e-03   3.502411e-08   cs_furtv
rf_rfcmp    4.744169e-03   1.680494e-56   rf_rfcmp
rf_verbl    1.415045e-02   1.82

In [17]:
printout.shape

(43, 3)

In [18]:
for i in range(43):
    if i != 2:
        lmone = smf.ols(formula = 'arstmade ~ {}'.format(sqf_primary_select.columns[i]), data = sqf_primary_select).fit()
        print 'arstmade ~ ' + sqf_primary_select.columns[i] + ' + 1'
        tempone = pd.DataFrame(lmone.params)
        temp2one = pd.DataFrame(lmone.pvalues)
        printout1 = pd.concat([tempone, temp2one], axis = 1)
        printout1.columns = ['params', 'p-value']
        print printout1

arstmade ~ explnstp + 1
             params        p-value
Intercept  0.070745  3.168447e-304
explnstp  -0.010737   1.606771e-08
arstmade ~ arstmade + 1
                 params  p-value
Intercept  1.095342e-14      0.0
arstmade   1.000000e+00      0.0
arstmade ~ frisked + 1
             params  p-value
Intercept  0.020291      0.0
frisked    0.076292      0.0
arstmade ~ searched + 1
             params  p-value
Intercept  0.022537      0.0
searched   0.443002      0.0
arstmade ~ contrabn + 1
             params  p-value
Intercept  0.046590      0.0
contrabn   0.732095      0.0
arstmade ~ rf_vcrim + 1
             params  p-value
Intercept  0.057518      0.0
rf_vcrim   0.022312      0.0
arstmade ~ rf_othsw + 1
             params  p-value
Intercept  0.048768      0.0
rf_othsw   0.153385      0.0
arstmade ~ rf_attir + 1
             params       p-value
Intercept  0.059707  0.000000e+00
rf_attir   0.004705  4.179812e-30
arstmade ~ cs_objcs + 1
             params  p-value
Intercept  0.05

In [19]:
#Correlation Matrix
sqf_primary_select.corr()

,explnstp,arstmade,sumissue,frisked,searched,contrabn,rf_vcrim,rf_othsw,rf_attir,cs_objcs,...,sex_Z,race_A,race_B,race_I,race_P,race_Q,race_U,race_W,race_X,race_Z
explnstp,1.000000,-0.002524,0.002540,0.013657,0.000028,-0.003432,0.005239,-0.028214,0.006094,-0.000797,...,-0.034568,-0.000051,0.007319,0.000672,0.001102,0.003174,0.000962,0.002460,-0.003343,-0.031734
arstmade,-0.002524,1.000000,-0.033154,0.160429,0.519106,0.413880,0.029748,0.168486,0.005094,0.069966,...,-0.015771,0.001632,-0.004774,-0.003157,0.007737,0.004179,-0.000334,0.004451,0.000557,-0.013999
sumissue,0.002540,-0.033154,1.000000,0.029084,0.032425,0.040696,-0.032872,0.005562,0.003968,0.000550,...,-0.006715,0.001773,-0.004116,0.000910,0.002792,0.006901,-0.003005,-0.002253,-0.001448,-0.005427
frisked,0.013657,0.160429,0.029084,1.000000,0.260820,0.086086,0.340677,0.079796,0.264440,-0.019025,...,-0.023402,-0.034819,0.044012,-0.011183,0.026408,0.018409,-0.007530,-0.081302,-0.005505,-0.024224
searched,0.000028,0.519106,0.032425,0.260820,1.000000,0.280054,0.058608,0.162653,0.051098,0.054773,...,-0.012339,-0.002148,-0.006277,-0.002637,0.006431,0.011671,-0.001428,-0.002166,-0.001247,-0.011086
contrabn,-0.003432,0.413880,0.040696,0.086086,0.280054,1.000000,-0.008354,0.081003,0.005508,0.053341,...,-0.007348,-0.003827,-0.002007,-0.003197,0.002409,-0.002248,-0.000937,0.012388,-0.000163,-0.007259
rf_vcrim,0.005239,0.029748,-0.032872,0.340677,0.058608,-0.008354,1.000000,-0.026465,0.049240,-0.018852,...,-0.017374,-0.003800,0.019536,-0.004479,0.004877,0.012063,-0.004777,-0.036509,-0.002251,-0.020466
rf_othsw,-0.028214,0.168486,0.005562,0.079796,0.162653,0.081003,-0.026465,1.000000,-0.025546,0.027779,...,0.028204,-0.003891,-0.001231,-0.002894,-0.003443,-0.003120,-0.008544,-0.004961,0.006407,0.029156
rf_attir,0.006094,0.005094,0.003968,0.264440,0.051098,0.005508,0.049240,-0.025546,1.000000,0.002766,...,-0.009107,-0.008342,0.020757,-0.004387,0.004734,0.002343,-0.002075,-0.028730,-0.003315,-0.010541
cs_objcs,-0.000797,0.069966,0.000550,-0.019025,0.054773,0.053341,-0.018852,0.027779,0.002766,1.000000,...,-0.000299,0.006306,-0.025375,0.002356,-0.001176,0.011588,0.000803,0.022583,0.001044,-0.000372


In [44]:
x1 = sqf_primary_select.iloc[:, 0:1]
x2 = sqf_primary_select.iloc[:, 2:]
X = pd.concat([x1, x2], axis = 1)

In [49]:
y = sqf_primary_select.arstmade


In [50]:
import numpy as np
import pandas as pd

def train_validate_test_split(df, train_percent=.33, validate_percent=.66, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.ix[perm[:train_end]]
    validate = df.ix[perm[train_end:validate_end]]
    test = df.ix[perm[validate_end:]]
    return train, validate, test

In [51]:
#train, validate, test = train_validate_test_split(sqf_primary_select)

In [52]:
from sklearn.cross_validation import train_test_split

In [53]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4)

In [54]:
##Logistic Regression

#learn logistic regression over the training sample
logit = LogisticRegression(C=1e6)
logit.fit(np.c_[X_train], y_train)
# C here controls the regularization strength--which will be discussed in Bayesian session
# For now just keep in mind that setting a high value will make the regularization ineffective


LogisticRegression(C=1000000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0)

In [55]:
#apply to the test sample
labelPred = logit.predict(np.c_[X_test])
labelActual = y_test

In [56]:
#compute TP,FP,FN,TN for the above classification
TP = sum((labelPred==1)&(labelActual==1))
FP = sum((labelPred==1)&(labelActual==0))
FN = sum((labelPred==0)&(labelActual==1))
TN = sum((labelPred==0)&(labelActual==0))
print('\nTP = %d, TN = %d, FP = %d, FN = %d'%(TP,TN,FP,FN))
#compute precision, accuracy and recall
Acc = 100.0 * (TP+TN)/(TP+TN+FP+FN)
Rec = 100.0 * TP/(TP+FN)
Prec= 100.0 * TP/(TP+FP)
print('Precision = %.2f%%\nAccuracy = %.2f%%\nRecall = %.2f%%'%(Prec,Acc,Rec))


TP = 68876, TN = 1865986, FP = 17000, FN = 51625
Precision = 80.20%
Accuracy = 96.57%
Recall = 57.16%


In [68]:
#print "coefficients of logistic regression: \n", 
logistic = []
#print "Intercept ", + logit.intercept_[0]
logistic.append(["Intercept", logit.intercept_[0]])
for i in range(X.shape[1]):
    logistic.append([X.columns[i], logit.coef_[0][i]])
    #print X.columns[i], + logit.coef_[0][i]


In [69]:
logistic = pd.DataFrame(logistic)
logistic.columns = ['var', 'logistic params']
#print logistic

In [70]:
loglinreg = logistic.merge(printout, on ="var")

In [71]:
loglinreg

,var,logistic params,linear params,linearpvalue
0,Intercept,-2.110725,-1.183776e+08,6.270085e-01
1,explnstp,-0.064601,-2.176676e-03,1.230802e-01
2,sumissue,-2.012436,-4.943126e-02,0.000000e+00
3,frisked,1.033593,3.343392e-02,0.000000e+00
4,searched,3.085349,3.892120e-01,0.000000e+00
5,contrabn,3.866663,4.123669e-01,0.000000e+00
6,rf_vcrim,-0.175891,-9.187001e-03,4.499973e-222
7,rf_othsw,0.067988,1.063812e-02,4.530451e-214
8,rf_attir,-0.222388,-1.170147e-02,2.002078e-270
9,cs_objcs,0.839828,3.688222e-02,0.000000e+00


In [109]:
df = pd.DataFrame(loglinreg.linearpvalue.nsmallest(10))

In [116]:
df = df.reset_index()

In [120]:
loglinreg = loglinreg.reset_index()
print loglinreg.head()

   level_0  index        var  logistic params  linear params  linearpvalue
0        0      0  Intercept        -2.110725  -1.183776e+08      0.627008
1        1      1   explnstp        -0.064601  -2.176676e-03      0.123080
2        2      2   sumissue        -2.012436  -4.943126e-02      0.000000
3        3      3    frisked         1.033593   3.343392e-02      0.000000
4        4      4   searched         3.085349   3.892120e-01      0.000000


In [128]:
joinedminp = df.merge(loglinreg, on = 'index', how = 'inner')
print "variables with minimum p values from linear regression are:"
print joinedminp['var']

variables with minimum p values from linear regression are:
0    sumissue
1     frisked
2    searched
3    contrabn
4    cs_objcs
5    cs_descr
6    cs_casng
7    cs_drgtr
8    cs_bulge
9    sb_hdobj
Name: var, dtype: object


In [129]:
df2 = loglinreg.linearpvalue.nlargest(10)
df2 = df2.reset_index()

In [130]:
joinedmaxp = df2.merge(loglinreg, on = 'index', how = 'inner')
print "variables with maximum p values from linear regression are:"
print joinedmaxp['var']

variables with maximum p values from linear regression are:
0      inout_I
1      inout_O
2    Intercept
3       race_Z
4       race_B
5       race_Q
6       race_I
7       race_U
8       race_W
9       race_X
Name: var, dtype: object
